## Pyspark for modeling STRR with H2O+Sparkling

In [24]:
# import s3fs
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
# csv file
# df_all = pd.read_csv("s3://eqrs-ngmc-datascience/Datascience/x.csv")
df_all = pd.read_csv("c:/project_data/x_strr.csv")

# parquet file
# df = pd.read_parquet('s3://{bucket_name}/{path_to_file}')

import warnings
warnings.filterwarnings('ignore')

In [25]:
df = df_all[:50000]
df.head()

,Unnamed: 0,X,PTNT_MSR_RISK_CHRCTRSTC_ID,ptnt_id,provfs,t_trans,trans_dar,pstrr,wt_trans,ot_trans,pdiab,pdismiss,pnhprev,logbmi,bmi_msg,agecat6,year,pyf_period_esrd,t_start,t_stop,ashd1,othcardiac1,carfail,noambul,pulmon,notrans,cancer,diabetes,pvasc,cva,smoke,alcoh,drug,inci_one,inci_miss,strr_pyf_start,END_DT,PRVDR_ID,DAY_AT_HOSP_CNT,MISG_FRQNT_CMRBDTY_IND,FRQNT_CMRBDTY_TXT,MSR_TYPE_ID,FEML_IND,trans_yar,strr_period
0,1,1,16877082,6216849,262506,0,70,1,1,4.248495,1,0,0,3.133060,0,0,2018,1,0.25,0.5,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,23-OCT-18,31-DEC-18,2848.0,NaN,NaN,NaN,15,NaN,0.191781,1
1,2,2,16877086,6216881,342562,0,51,1,1,3.931826,0,0,0,2.555747,0,6,2018,1,0.25,0.5,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,11-NOV-18,31-DEC-18,6375.0,NaN,NaN,NaN,15,NaN,0.139726,1
2,3,3,16877134,1963204,552733,1,110,1,1,4.700480,1,0,0,3.251380,0,6,2017,6,5.00,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,13-SEP-17,31-DEC-17,4461.0,NaN,NaN,NaN,15,NaN,0.301370,1
3,4,4,16877135,1963204,552733,0,59,1,1,4.077537,1,0,0,3.251380,0,6,2018,6,5.00,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,01-JAN-18,28-FEB-18,4461.0,NaN,NaN,NaN,15,NaN,0.161644,1
4,5,5,16877136,1963204,552733,0,91,0,1,4.510860,1,0,0,3.251380,0,6,2018,6,5.00,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,01-APR-18,30-JUN-18,4461.0,NaN,NaN,NaN,15,NaN,0.249315,1


In [26]:
# as.Date(data$strr_pyf_start, format="%m/%d/%Y")
df['strr_pyf_start'] = pd.to_datetime(df['strr_pyf_start']).dt.strftime('%m/%d/%Y')
# df['provfs'] = df['provfs'].astype(str)
df['provfs'] = pd.to_numeric(df['provfs'], downcast='integer', errors='coerce').fillna(0)
df['provfs'] = df['provfs'].astype(int)

In [27]:
col_names = df.columns
df["provfs"].loc[0]

262506

In [28]:
# sorting with three columns
z_pd = df.to_records()
z_pd.sort(order=["provfs", "ptnt_id", "year"])
d2 = pd.DataFrame(z_pd)
d2

,index,Unnamed: 0,X,PTNT_MSR_RISK_CHRCTRSTC_ID,ptnt_id,provfs,t_trans,trans_dar,pstrr,wt_trans,ot_trans,pdiab,pdismiss,pnhprev,logbmi,bmi_msg,agecat6,year,pyf_period_esrd,t_start,t_stop,ashd1,othcardiac1,carfail,noambul,pulmon,notrans,cancer,diabetes,pvasc,cva,smoke,alcoh,drug,inci_one,inci_miss,strr_pyf_start,END_DT,PRVDR_ID,DAY_AT_HOSP_CNT,MISG_FRQNT_CMRBDTY_IND,FRQNT_CMRBDTY_TXT,MSR_TYPE_ID,FEML_IND,trans_yar,strr_period
0,7540,7541,7541,16893349,915,0,1,92,0,1,4.521789,1,0,1,3.100047,0,6,2016,6,5.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,10/01/2016,31-DEC-16,14033.0,NaN,NaN,NaN,15,NaN,0.252055,1
1,7541,7542,7542,16893350,915,0,0,59,1,1,4.077537,1,0,1,3.100047,0,6,2017,6,5.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,01/01/2017,28-FEB-17,14033.0,NaN,NaN,NaN,15,NaN,0.161644,1
2,7576,7577,7577,16893351,915,0,0,61,0,1,4.110874,1,0,1,3.100047,0,6,2017,6,5.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,04/01/2017,31-MAY-17,14033.0,NaN,NaN,NaN,15,NaN,0.167123,1
3,22116,22117,22117,16925468,2418,0,2,122,1,2,4.110874,1,0,0,3.166910,0,0,2017,5,3.0,5.0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,09/01/2017,31-DEC-17,9574.0,NaN,NaN,NaN,15,NaN,0.334247,1
4,22117,22118,22118,16925469,2418,0,0,31,1,1,3.433987,1,0,1,3.166910,0,0,2018,5,3.0,5.0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,01/01/2018,31-JAN-18,9574.0,NaN,NaN,NaN,15,NaN,0.084932,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,37692,37693,37693,16959744,331234,852540,1,143,1,1,4.962845,0,0,0,3.140479,0,0,2018,5,3.0,5.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,03/11/2018,31-JUL-18,53257.0,NaN,NaN,NaN,15,NaN,0.391781,1
49996,37693,37694,37694,16959745,331234,852540,1,153,0,1,5.030438,0,0,0,3.140479,0,0,2018,6,5.0,6.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,08/01/2018,31-DEC-18,53257.0,NaN,NaN,NaN,15,NaN,0.419178,1
49997,35112,35113,35113,16956968,1967490,852540,0,50,1,1,3.912023,1,0,0,3.361344,0,0,2018,4,2.0,3.0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,03/12/2018,30-APR-18,53257.0,NaN,NaN,NaN,15,NaN,0.136986,1
49998,35113,35114,35114,16956969,1967490,852540,0,31,0,1,3.433987,1,0,0,3.361344,0,0,2018,5,3.0,5.0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,12/01/2018,31-DEC-18,53257.0,NaN,NaN,NaN,15,NaN,0.084932,1


In [30]:
  classnames = (
    "pdiab", "pdismiss", "pnhprev", "bmi_msg", "ashd1", "othcardiac1",
    "carfail", "noambul", "pulmon", "notrans", "cancer", "diabetes",
    "pvasc", "cva", "smoke", "alcoh", "drug", "inci_one", "inci_miss"
  )

In [31]:
z = [ list(d2.columns).index(x) if x in d2.columns else None for x in  classnames]

In [32]:


for i in range(0, len(z)):
    col_mean = float(d2.iloc[:, [z[i]]].mean())
    if(col_mean<0.001 or col_mean>0.999):
        print("WARNING: Covariate", classnames[i],"has less than 0.1% variation.  The distribution of this covariate is", col_mean)
    else:
        print("WARNING: Covariate", classnames[i], "has missing values.")

    

In [33]:

cols = ["ptnt_id", "provfs", "t_trans", "strr_pyf_start", "wt_trans", "ot_trans","pdiab", 
        "pdismiss", "pnhprev",  "logbmi", "bmi_msg", "agecat6", "year", "pyf_period_esrd",  
        "t_start", "t_stop", "ashd1", "othcardiac1", "carfail", "noambul",  "pulmon", 
        "notrans", "cancer", "diabetes", "pvasc", "cva",  "smoke", "alcoh", "drug", "inci_one", "inci_miss"]
z2 = d2[cols]
z2

,ptnt_id,provfs,t_trans,strr_pyf_start,wt_trans,ot_trans,pdiab,pdismiss,pnhprev,logbmi,bmi_msg,agecat6,year,pyf_period_esrd,t_start,t_stop,ashd1,othcardiac1,carfail,noambul,pulmon,notrans,cancer,diabetes,pvasc,cva,smoke,alcoh,drug,inci_one,inci_miss
0,915,0,1,10/01/2016,1,4.521789,1,0,1,3.100047,0,6,2016,6,5.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
1,915,0,0,01/01/2017,1,4.077537,1,0,1,3.100047,0,6,2017,6,5.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
2,915,0,0,04/01/2017,1,4.110874,1,0,1,3.100047,0,6,2017,6,5.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
3,2418,0,2,09/01/2017,2,4.110874,1,0,0,3.166910,0,0,2017,5,3.0,5.0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0
4,2418,0,0,01/01/2018,1,3.433987,1,0,1,3.166910,0,0,2018,5,3.0,5.0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,331234,852540,1,03/11/2018,1,4.962845,0,0,0,3.140479,0,0,2018,5,3.0,5.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
49996,331234,852540,1,08/01/2018,1,5.030438,0,0,0,3.140479,0,0,2018,6,5.0,6.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
49997,1967490,852540,0,03/12/2018,1,3.912023,1,0,0,3.361344,0,0,2018,4,2.0,3.0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0
49998,1967490,852540,0,12/01/2018,1,3.433987,1,0,0,3.361344,0,0,2018,5,3.0,5.0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0


In [34]:
missing_z2 = z2.isna()

In [35]:

if sum(missing_z2.sum(axis=0))>0:
  print("Warning: There are", sum(missing_z2.sum(axis=0)), "rows with missing data")
  print("Any rows with missing data will be deleted")


In [36]:

allnames= ["ptnt_id","provfs","t_trans", "strr_pyf_start","wt_trans","ot_trans","pdiab","pdismiss",
    "pnhprev","logbmi","bmi_msg","agecat6","year","pyf_period_esrd","t_start","t_stop","ashd1", 
    "othcardiac1", "carfail","noambul", "pulmon", "notrans", "cancer", "diabetes",
    "pvasc", "cva", "smoke","alcoh", "drug","inci_one","inci_miss",
    "strr_period", "pstrr", "trans_yar", "trans_dar"]


data_sub = d2[allnames]
data_sub_complete=data_sub
data_sub_complete

,ptnt_id,provfs,t_trans,strr_pyf_start,wt_trans,ot_trans,pdiab,pdismiss,pnhprev,logbmi,bmi_msg,agecat6,year,pyf_period_esrd,t_start,t_stop,ashd1,othcardiac1,carfail,noambul,pulmon,notrans,cancer,diabetes,pvasc,cva,smoke,alcoh,drug,inci_one,inci_miss,strr_period,pstrr,trans_yar,trans_dar
0,915,0,1,10/01/2016,1,4.521789,1,0,1,3.100047,0,6,2016,6,5.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0.252055,92
1,915,0,0,01/01/2017,1,4.077537,1,0,1,3.100047,0,6,2017,6,5.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0.161644,59
2,915,0,0,04/01/2017,1,4.110874,1,0,1,3.100047,0,6,2017,6,5.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0.167123,61
3,2418,0,2,09/01/2017,2,4.110874,1,0,0,3.166910,0,0,2017,5,3.0,5.0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0.334247,122
4,2418,0,0,01/01/2018,1,3.433987,1,0,1,3.166910,0,0,2018,5,3.0,5.0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0.084932,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,331234,852540,1,03/11/2018,1,4.962845,0,0,0,3.140479,0,0,2018,5,3.0,5.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0.391781,143
49996,331234,852540,1,08/01/2018,1,5.030438,0,0,0,3.140479,0,0,2018,6,5.0,6.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0.419178,153
49997,1967490,852540,0,03/12/2018,1,3.912023,1,0,0,3.361344,0,0,2018,4,2.0,3.0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,1,0.136986,50
49998,1967490,852540,0,12/01/2018,1,3.433987,1,0,0,3.361344,0,0,2018,5,3.0,5.0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0.084932,31


In [37]:
########################################################################################
#  SECTION 7 (ISSUE #3): CREATE TRANSFUSION EVENT FLAG                                 #
#    -CHECK FOR SMALL NUMBER OF EVENTS                                                 #                                           
########################################################################################

data_sub_complete['t_trans0'] = np.where(data_sub_complete['t_trans']>0, 1, 0)
data_sub_complete['t_trans']=data_sub_complete['t_trans'].apply(lambda x: 1 if x > 0 else 0).copy()
# data_sub_complete.loc[data_sub_complete.t_trans > 1, 't_trans'] = 1
# data_sub_complete.loc[data_sub_complete.t_trans <= 0, 't_trans'] = 0
data_sub_complete

,ptnt_id,provfs,t_trans,strr_pyf_start,wt_trans,ot_trans,pdiab,pdismiss,pnhprev,logbmi,bmi_msg,agecat6,year,pyf_period_esrd,t_start,t_stop,ashd1,othcardiac1,carfail,noambul,pulmon,notrans,cancer,diabetes,pvasc,cva,smoke,alcoh,drug,inci_one,inci_miss,strr_period,pstrr,trans_yar,trans_dar,t_trans0
0,915,0,1,10/01/2016,1,4.521789,1,0,1,3.100047,0,6,2016,6,5.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0.252055,92,1
1,915,0,0,01/01/2017,1,4.077537,1,0,1,3.100047,0,6,2017,6,5.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0.161644,59,0
2,915,0,0,04/01/2017,1,4.110874,1,0,1,3.100047,0,6,2017,6,5.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0.167123,61,0
3,2418,0,1,09/01/2017,2,4.110874,1,0,0,3.166910,0,0,2017,5,3.0,5.0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0.334247,122,1
4,2418,0,0,01/01/2018,1,3.433987,1,0,1,3.166910,0,0,2018,5,3.0,5.0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0.084932,31,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,331234,852540,1,03/11/2018,1,4.962845,0,0,0,3.140479,0,0,2018,5,3.0,5.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0.391781,143,1
49996,331234,852540,1,08/01/2018,1,5.030438,0,0,0,3.140479,0,0,2018,6,5.0,6.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0.419178,153,1
49997,1967490,852540,0,03/12/2018,1,3.912023,1,0,0,3.361344,0,0,2018,4,2.0,3.0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,1,0.136986,50,0
49998,1967490,852540,0,12/01/2018,1,3.433987,1,0,0,3.361344,0,0,2018,5,3.0,5.0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0.084932,31,0


In [38]:
sum(data_sub_complete['provfs']==0)

51

In [39]:
########################################################################################
#   SECTION 8: SUBSET DATA TO THOSE WITH AT LEAST 1 DAY AT RISK                        #
#     -ALSO SUBSET TO THOSE WITH APPROPRIATE AGE AND ESRD CATEGORIES                   #
#   NOTE: THIS IS MOSTLY DONE JUST TO BE PRECAUTIOUS FOR TEST DATA                     #
########################################################################################

data_sub_complete2 = data_sub_complete[(data_sub_complete['pyf_period_esrd']>0) & (data_sub_complete['agecat6']!=1) & (data_sub_complete['trans_dar']>0)]
data_sub_complete2

,ptnt_id,provfs,t_trans,strr_pyf_start,wt_trans,ot_trans,pdiab,pdismiss,pnhprev,logbmi,bmi_msg,agecat6,year,pyf_period_esrd,t_start,t_stop,ashd1,othcardiac1,carfail,noambul,pulmon,notrans,cancer,diabetes,pvasc,cva,smoke,alcoh,drug,inci_one,inci_miss,strr_period,pstrr,trans_yar,trans_dar,t_trans0
0,915,0,1,10/01/2016,1,4.521789,1,0,1,3.100047,0,6,2016,6,5.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0.252055,92,1
1,915,0,0,01/01/2017,1,4.077537,1,0,1,3.100047,0,6,2017,6,5.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0.161644,59,0
2,915,0,0,04/01/2017,1,4.110874,1,0,1,3.100047,0,6,2017,6,5.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0.167123,61,0
3,2418,0,1,09/01/2017,2,4.110874,1,0,0,3.166910,0,0,2017,5,3.0,5.0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0.334247,122,1
4,2418,0,0,01/01/2018,1,3.433987,1,0,1,3.166910,0,0,2018,5,3.0,5.0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0.084932,31,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,331234,852540,1,03/11/2018,1,4.962845,0,0,0,3.140479,0,0,2018,5,3.0,5.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0.391781,143,1
49996,331234,852540,1,08/01/2018,1,5.030438,0,0,0,3.140479,0,0,2018,6,5.0,6.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0.419178,153,1
49997,1967490,852540,0,03/12/2018,1,3.912023,1,0,0,3.361344,0,0,2018,4,2.0,3.0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,1,0.136986,50,0
49998,1967490,852540,0,12/01/2018,1,3.433987,1,0,0,3.361344,0,0,2018,5,3.0,5.0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0.084932,31,0


In [40]:
########################################################################################
#   SECTION 9 (ISSUE #5): CHECK FOR LINEARLY DEPENDENT COVARIATES                      #
#      NOTE: THIS WON'T PREVENT MODEL FROM RUNNING                                     #
########################################################################################

z_pd = data_sub_complete2.to_records()
z_pd.sort(order=["provfs", "ptnt_id", "year", "strr_pyf_start"])
data_sub_sort = pd.DataFrame(z_pd)
data_chk_rank = data_sub_sort.drop(["strr_pyf_start","provfs","ptnt_id"], axis = 1) 
data_chk_rank

,index,t_trans,wt_trans,ot_trans,pdiab,pdismiss,pnhprev,logbmi,bmi_msg,agecat6,year,pyf_period_esrd,t_start,t_stop,ashd1,othcardiac1,carfail,noambul,pulmon,notrans,cancer,diabetes,pvasc,cva,smoke,alcoh,drug,inci_one,inci_miss,strr_period,pstrr,trans_yar,trans_dar,t_trans0
0,0,1,1,4.521789,1,0,1,3.100047,0,6,2016,6,5.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0.252055,92,1
1,1,0,1,4.077537,1,0,1,3.100047,0,6,2017,6,5.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0.161644,59,0
2,2,0,1,4.110874,1,0,1,3.100047,0,6,2017,6,5.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0.167123,61,0
3,3,1,2,4.110874,1,0,0,3.166910,0,0,2017,5,3.0,5.0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0.334247,122,1
4,4,0,1,3.433987,1,0,1,3.166910,0,0,2018,5,3.0,5.0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0.084932,31,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,1,1,4.962845,0,0,0,3.140479,0,0,2018,5,3.0,5.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0.391781,143,1
49996,49996,1,1,5.030438,0,0,0,3.140479,0,0,2018,6,5.0,6.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0.419178,153,1
49997,49997,0,1,3.912023,1,0,0,3.361344,0,0,2018,4,2.0,3.0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,1,0.136986,50,0
49998,49998,0,1,3.433987,1,0,0,3.361344,0,0,2018,5,3.0,5.0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0.084932,31,0


In [41]:
########################################################################################
#   SECTION 10: RUN DATA THROUGH TWO COX PROPORTIONAL HAZARDS MODELS                   #
########################################################################################


data_sub_sort['year'] = pd.Categorical(data_sub_sort['year'], ordered=False)
data_sub_sort['pdismiss'] = pd.Categorical(data_sub_sort['pdismiss'], categories=[0, 1], ordered=False)
data_sub_sort['pnhprev'] = pd.Categorical(data_sub_sort['pnhprev'], categories=[0, 1], ordered=False)
data_sub_sort['bmi_msg'] = pd.Categorical(data_sub_sort['bmi_msg'], categories=[0, 1], ordered=False)
data_sub_sort['ashd1'] = pd.Categorical(data_sub_sort['ashd1'], categories=[0, 1], ordered=False)
data_sub_sort['othcardiac1'] = pd.Categorical(data_sub_sort['othcardiac1'], categories=[0, 1], ordered=False)
data_sub_sort['carfail'] = pd.Categorical(data_sub_sort['carfail'], categories=[0, 1], ordered=False)
data_sub_sort['noambul'] = pd.Categorical(data_sub_sort['noambul'], categories=[0, 1], ordered=False)
data_sub_sort['pulmon'] = pd.Categorical(data_sub_sort['pulmon'], categories=[0, 1], ordered=False)
data_sub_sort['notrans'] = pd.Categorical(data_sub_sort['notrans'], categories=[0, 1], ordered=False)
data_sub_sort['cancer'] = pd.Categorical(data_sub_sort['cancer'], categories=[0, 1], ordered=False)
data_sub_sort['diabetes'] = pd.Categorical(data_sub_sort['diabetes'], categories=[0, 1], ordered=False)
data_sub_sort['pvasc'] = pd.Categorical(data_sub_sort['pvasc'], categories=[0, 1], ordered=False)
data_sub_sort['cva'] = pd.Categorical(data_sub_sort['cva'], categories=[0, 1], ordered=False)
data_sub_sort['smoke'] = pd.Categorical(data_sub_sort['smoke'], categories=[0, 1], ordered=False)
data_sub_sort['alcoh'] = pd.Categorical(data_sub_sort['alcoh'], categories=[0, 1], ordered=False)
data_sub_sort['drug'] = pd.Categorical(data_sub_sort['drug'], categories=[0, 1], ordered=False)
data_sub_sort['inci_one'] = pd.Categorical(data_sub_sort['inci_one'], categories=[0, 1], ordered=False)
data_sub_sort['inci_miss'] = pd.Categorical(data_sub_sort['inci_miss'], categories=[0, 1], ordered=False)
data_sub_sort['smoke'] = pd.Categorical(data_sub_sort['smoke'], categories=[0, 1], ordered=False)

data_sub_sort['provfs'] = pd.Categorical(data_sub_sort['provfs'], ordered=False)

# coxph_control <- coxph.control(eps = 1e-8)
# data_sub_sort.transform(lambda x: x + 1)

data_model = data_sub_sort


In [43]:

#######################################################################################################################################################
# Run Stage-1 Cox Model
#######################################################################################################################################################



#############################################
import h2o
from h2o.estimators.coxph import H2OCoxProportionalHazardsEstimator
h2o.init()

data_modelt_Full_Sample=data_model
data_modelt_Full_Sample.reset_index(drop=True, inplace=True)

data_modelt_Full_Sample_hex = h2o.H2OFrame(data_modelt_Full_Sample)
data_modelt_Full_Sample_hex[data_modelt_Full_Sample_hex["provfs"].isna(), "provfs"] = 0
# data_modelt_Full_Sample_hex["provfs"] = data_modelt_Full_Sample_hex["provfs"].ascharacter()
# data_modelt_Full_Sample_hex["provfs"] = data_modelt_Full_Sample_hex["provfs"].asfactor()
predictorsSt = list(["agecat6", "pdiab", "pdismiss","notrans","cancer",
                "diabetes","pvasc","year", "pnhprev", "logbmi", 
                "bmi_msg","cva","smoke","alcoh","drug",
                "inci_one", "ashd1", "pulmon","inci_miss","year",
                "othcardiac1", "carfail", "noambul", "pulmon"])

interaction_pairs = [   ("agecat6","pdiab"), 
                        ("pdiab", "pyf_period_esrd")]

strr_h2o_moodel = H2OCoxProportionalHazardsEstimator(
    start_column="t_start",
    stop_column="t_stop",
    offset_column="ot_trans",
    ties="breslow",
#     stratify_by=["provfs"],
#     interaction_pairs=interaction_pairs,
    )

strr_h2o_moodel.train(x=predictorsSt,
                y="t_trans0",
                training_frame=data_modelt_Full_Sample_hex)
                        
# coefficients_Stage_1<-strr_h2o_moodel.model.coefficients_table.coefficients

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,3 hours 50 mins
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,2 months and 22 days
H2O_cluster_name:,H2O_from_python_IZabett_n3czrb
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.353 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
coxph Model Build progress: |█████████████████████████████████████████████| 100%


In [ ]:
# writing to csv
pd.DataFrame(data_modelt_Full_Sample_hex["provfs"]).to_csv('provfs.csv')

In [44]:
strr_h2o_moodel.coefficients_table

Model Details
H2OCoxProportionalHazardsEstimator :  Cox Proportional Hazards
Model Key:  CoxPH_model_python_1608552908847_3

Call: 
Surv(t_start, t_stop, t_trans0) ~ pdiab + pdismiss + pnhprev + logbmi + bmi_msg + agecat6 + year + ashd1 + othcardiac1 + carfail + noambul + pulmon + notrans + cancer + diabetes + pvasc + cva + smoke + alcoh + drug + inci_one + inci_miss + offset(ot_trans)

Coefficients: CoxPH Coefficients


,names,coefficients,exp_coef,exp_neg_coef,se_coef,z_coef
0,pdiab,0.099197,1.104284,0.905564,0.044976,2.205559
1,pdismiss,0.278771,1.321505,0.756713,0.171733,1.623277
2,pnhprev,0.469407,1.599046,0.625373,0.034871,13.461366
3,logbmi,-0.107639,0.897951,1.113646,0.054310,-1.981962
4,bmi_msg,0.044401,1.045401,0.956570,0.117583,0.377615
5,agecat6,0.008895,1.008935,0.991144,0.005800,1.533669
6,year,-0.046274,0.954780,1.047362,0.012561,-3.683932
7,ashd1,0.077683,1.080780,0.925258,0.043164,1.799702
8,othcardiac1,0.032465,1.032998,0.968056,0.035543,0.913414
9,carfail,0.065894,1.068113,0.936231,0.033124,1.989318



See the whole table with table.as_data_frame()

Likelihood ratio test=376.457503
n=50000, number of events=5284

Scoring History: 


,,timestamp,duration,iterations,loglik
0,,2020-12-21 11:05:43,0.000 sec,0,-50231.922822
1,,2020-12-21 11:05:43,0.149 sec,1,-50050.670264
2,,2020-12-21 11:05:43,0.283 sec,2,-50043.700396
3,,2020-12-21 11:05:43,0.410 sec,3,-50043.694071


<bound method H2OCoxPHModel.coefficients_table of >

In [ ]:
h2o.shutdown()